In [1]:
f1 :: Num a => a -> a
f1 = \x -> x - 2

Line 2: Redundant lambda
Found:
f1 = \ x -> x - 2
Why not:
f1 x = x - 2

In [2]:
f2 :: Floating a => (a, a) -> a
f2 = \(x, y) -> sqrt(x^2 + y^2)

Line 2: Redundant lambda
Found:
f2 = \ (x, y) -> sqrt (x ^ 2 + y ^ 2)
Why not:
f2 (x, y) = sqrt (x ^ 2 + y ^ 2)

In [14]:
isqrt :: Int -> Int
isqrt = floor.sqrt.fromIntegral
f3 :: (Int, Int, Int) -> Int
f3 = \(x, y, z) -> isqrt(x^2 + y^2 + z^2)

Line 4: Redundant lambda
Found:
f3 = \ (x, y, z) -> isqrt (x ^ 2 + y ^ 2 + z ^ 2)
Why not:
f3 (x, y, z) = isqrt (x ^ 2 + y ^ 2 + z ^ 2)

In [6]:
f7 :: Int -> Bool
f7 = \x -> x `mod` 2 == 0

Line 2: Redundant lambda
Found:
f7 = \ x -> x `mod` 2 == 0
Why not:
f7 x = x `mod` 2 == 0

In [16]:
f8 :: Floating a => a -> a
f8 x = (\y -> (y $ sqrt x))
        (\y -> 2 * y^3 * (y + 1))

Line 2: Redundant bracket
Found:
\ y -> (y $ sqrt x)
Why not:
\ y -> y $ sqrt x

In [8]:
f9 :: Int -> Int
f9 = \x -> case x of
            1 -> 3
            _ -> 0

Line 2: Redundant lambda
Found:
f9
  = \ x ->
      case x of
          1 -> 3
          _ -> 0
Why not:
f9 x
  = case x of
        1 -> 3
        _ -> 0Line 2: Use lambda-case
Found:
\ x ->
  case x of
      1 -> 3
      _ -> 0
Why not:
\case
    1 -> 3
    _ -> 0

In [19]:
f10 :: Int -> Int
f10 = \x -> 2*x
f10 5

Line 2: Redundant lambda
Found:
f10 = \ x -> 2 * x
Why not:
f10 x = 2 * xLine 2: Avoid lambda
Found:
\ x -> 2 * x
Why not:
(2 *)

10

In [20]:
sum' :: Num a => [a] -> a
sum' []     = 0
sum' (x:xs) = x + sum' xs

Line 2: Use foldr
Found:
sum' [] = 0
sum' (x : xs) = x + sum' xs
Why not:
sum' xs = foldr (+) 0 xs

In [21]:
sumWith :: Num a => (a -> a) -> [a] -> a
sumWith _ [] = 0
sumWith f (x:xs) = f x + sumWith f xs

In [22]:
sum' :: Num a => [a] -> a
sum' = sumWith(\e -> e)

Line 2: Use id
Found:
\ e -> e
Why not:
id

In [23]:
sumSqr' :: Num a => [a] -> a
sumSqr' = sumWith(\e -> e^2)

Line 2: Avoid lambda
Found:
\ e -> e ^ 2
Why not:
(^ 2)

In [24]:
listLength :: Num a => [a] -> a
listLength = sumWith(\e -> 1)

Line 2: Use const
Found:
\ e -> 1
Why not:
const 1

In [33]:
expApproxUpTo :: Int -> Double -> Double
expApproxUpTo n = case n of 
    0 -> \x -> 1
    1 -> \x -> x + 1
    _ -> \x -> ((x^n) / fromIntegral (factorial n)) + expApproxUpTo (n-1) x
    where 
        factorial 0 = 1
        factorial k = k * factorial (k-1)

Line 3: Use const
Found:
\ x -> 1
Why not:
const 1Line 4: Avoid lambda
Found:
\ x -> x + 1
Why not:
(+ 1)

In [35]:
import Data.List
sortDesc :: Ord a => [a] -> [a]
sortDesc xs = (reverse . sort) xs

Line 3: Eta reduce
Found:
sortDesc xs = (reverse . sort) xs
Why not:
sortDesc = (reverse . sort)Line 3: Avoid reverse
Found:
reverse . sort
Why not:
sortOn Data.Ord.Down

In [37]:
import Data.List
sortDesc' :: Ord a => [a] -> [a]
sortDesc' = reverse . sort

Line 3: Avoid reverse
Found:
reverse . sort
Why not:
sortOn Data.Ord.Down

In [61]:
((,) $ 1 )$ 2

Line 1: Use tuple-section
Found:
((,) $ 1)
Why not:
(1,)Line 1: Redundant $
Found:
((,) $ 1) $ 2
Why not:
((,) $ 1) 2Line 1: Redundant $
Found:
(,) $ 1
Why not:
(,) 1

(1,2)

In [65]:
sumWith g [] = 0
sumWith g (x:xs) = g x + sumWith g xs

prodWith g [] = 1
prodWith g (x:xs) = g x * prodWith g xs

sumWith' :: Num a => (a -> a) -> [a] -> a
sumWith' = go 0
    where
        go acc g [] = acc
        go acc g (x:xs) = go (g x + acc) g xs

prodWith' :: Num a => (a -> a) -> [a] -> a
prodWith' = go 1
    where
        go acc g [] = acc
        go acc g (x:xs) = go (g x * acc) g xs
        
foldr' :: (a -> b -> b) -> b -> [a] -> b
foldr' f z [] = z
foldr' f z (x:xs) = f x (foldr' f z xs)
        
sumWith'' g = foldr' (\x acc -> g x + acc) 0
prodWith'' g = foldr' (\x acc -> g x * acc) 1


foldl' :: (b -> a -> b) -> b -> [a] -> b
foldl' f z [] = z
foldl' f z (x:xs) = foldl' f (f z x) xs

sumWith''' g = foldl' (\acc x -> g x + acc) 0
prodWith''' g = foldl' (\acc x -> g x * acc) 1


Line 1: Use foldr
Found:
sumWith g [] = 0
sumWith g (x : xs) = g x + sumWith g xs
Why not:
sumWith g xs = foldr ((+) . g) 0 xsLine 4: Use foldr
Found:
prodWith g [] = 1
prodWith g (x : xs) = g x * prodWith g xs
Why not:
prodWith g xs = foldr ((*) . g) 1 xsLine 10: Use foldl
Found:
go acc g [] = acc
go acc g (x : xs) = go (g x + acc) g xs
Why not:
go acc g xs = foldl (\ acc x -> g x + acc) acc xsLine 16: Use foldl
Found:
go acc g [] = acc
go acc g (x : xs) = go (g x * acc) g xs
Why not:
go acc g xs = foldl (\ acc x -> g x * acc) acc xsLine 20: Use foldr
Found:
foldr' f z [] = z
foldr' f z (x : xs) = f x (foldr' f z xs)
Why not:
foldr' f z xs = foldr f z xs

In [66]:
isSortedAsc :: Ord a => [a] -> Bool
isSortedAsc xs = length xs - 1 == (length $ filter (\(x,y) -> x <= y) $ zip xs (tail xs))


Line 2: Move brackets to avoid $
Found:
length xs - 1 ==
  (length $ filter (\ (x, y) -> x <= y) $ zip xs (tail xs))
Why not:
length xs - 1 ==
  length (filter (\ (x, y) -> x <= y) $ zip xs (tail xs))Line 2: Use uncurry
Found:
\ (x, y) -> x <= y
Why not:
uncurry (<=)

In [67]:
everySecond :: [t] -> [Int]
everySecond xs = map (\(x,y) -> y) $ filter(\(x,y) -> y `mod` 2 == 1) $ zip xs [1..length xs]


Line 2: Use snd
Found:
\ (x, y) -> y
Why not:
snd